## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [1]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

env: GOOGLE_COLAB=1
Mounted at /content/drive


# License Plate Recognition using TAO LPRNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained baseline18 LPRNet model and train it on the OpenALPR benchmark dataset
* Run Inference on the trained model
* Export the trained model to a .etlt file for deployment to DeepStream

## Table of Contents

This notebook shows an example usecase of LPRNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1.1 [Download pre-trained model](#head-1-1) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)

#### Note
1. This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly
2. This notebook uses OPENALPR dataset by default, which should be around ~2.2 MB.
3. Using the default config/spec file provided in this notebook, each weight file size of lprnet created during training will be ~111 MB

## 0. Set up env variables and set FIXME parameters <a class="anchor" id="head-0"></a>

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

#### FIXME
1. NUM_GPUS - set this to <= number of GPU's availble on the instance
1. GPU_INDEX - set to to the indices of the GPU available on the instance
1. COLAB_NOTEBOOKS_PATH - for Google Colab environment, set this path where you want to clone the repo to; for local system environment, set this path to the already cloned repo
1. EXPERIMENT_DIR - set this path to a folder location where pretrained models, checkpoints and log files during different model actions will be saved
1. delete_existing_experiments - set to True to remove existing pretrained models, checkpoints and log files of a previous experiment
1. DATA_DIR - set this path to a folder location where you want to dataset to be present
1. delete_existing_data - set this to True to remove existing preprocessed and original data

In [2]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1
#FIXME2
%env GPU_INDEX=0

#FIXME3
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/nvidia-tao
if os.environ["GOOGLE_COLAB"] == "1":
    if not os.path.exists(os.path.join(os.environ["COLAB_NOTEBOOKS_PATH"])):
      !git clone https://github.com/NVIDIA-AI-IOT/nvidia-tao.git $COLAB_NOTEBOOKS_PATH
else:
    if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
        raise Exception("Error, enter the path of the colab notebooks repo correctly")

#FIXME4
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/lprnet
#FIXME5
delete_existing_experiments = True
#FIXME6
%env DATA_DIR=/content/drive/MyDrive/lprnet_data/
#FIXME7
delete_existing_data = False

if delete_existing_experiments:
    !sudo rm -rf $EXPERIMENT_DIR
if delete_existing_data:
    !sudo rm -rf $DATA_DIR

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/lprnet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

env: TAO_DOCKER_DISABLE=1
env: KEY=nvidia_tlt
env: NUM_GPUS=1
env: GPU_INDEX=0
env: COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/nvidia-tao
Cloning into '/content/drive/MyDrive/nvidia-tao'...
remote: Enumerating objects: 2281, done.
remote: Counting objects: 100% (961/961), done.
remote: Compressing objects: 100% (579/579), done.
remote: Total 2281 (delta 614), reused 712 (delta 372), pack-reused 1320
Receiving objects: 100% (2281/2281), 3.99 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (1431/1431), done.
env: EXPERIMENT_DIR=/content/drive/MyDrive/results/lprnet
env: DATA_DIR=/content/drive/MyDrive/lprnet_data/
env: SPECS_DIR=/content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/specs
total 2
-rw------- 1 root root   70 Jan 30 08:19 us_lp_characters.txt
-rw------- 1 root root 1137 Jan 30 08:19 tutorial_spec.txt


## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

 We will be using the OpenALPR benchmark dataset for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO. 

In [3]:
!bash $COLAB_NOTEBOOKS_PATH/tensorflow/lprnet/download_and_prepare_data.sh $DATA_DIR

+ '[' -z /content/drive/MyDrive/lprnet_data/ ']'
++ pwd
+ CURRENT_DIR=/content
+ echo 'Cloning OpenALPR benchmark directory'
Cloning OpenALPR benchmark directory
+ '[' '!' -e benchmarks ']'
+ git clone https://github.com/openalpr/benchmarks benchmarks
Cloning into 'benchmarks'...
remote: Enumerating objects: 1752, done.
remote: Total 1752 (delta 0), reused 0 (delta 0), pack-reused 1752
Receiving objects: 100% (1752/1752), 187.98 MiB | 31.15 MiB/s, done.
Resolving deltas: 100% (34/34), done.
+ OUTPUT_DIR=/content/drive/MyDrive/lprnet_data
+ mkdir -p /content/drive/MyDrive/lprnet_data
+++ readlink -f /content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/download_and_prepare_data.sh
++ dirname /content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/download_and_prepare_data.sh
+ SCRIPT_DIR=/content/drive/MyDrive/nvidia-tao/tensorflow/lprnet
+ echo 'Preprocessing OpenALPR benchmarks data for US'
Preprocessing OpenALPR benchmarks data for US
+ python3 /content/drive/MyDrive/nvidia-tao/tensorflow/

In [4]:
# verify
!echo $DATA_DIR
!ls -l $DATA_DIR/
!ls -l $DATA_DIR/train
!ls -l $DATA_DIR/train/image
!ls -l $DATA_DIR/train/label


/content/drive/MyDrive/lprnet_data/
total 8
drwx------ 4 root root 4096 Jan 30 08:20 train
drwx------ 4 root root 4096 Jan 30 08:20 val
total 8
drwx------ 2 root root 4096 Jan 30 08:20 image
drwx------ 2 root root 4096 Jan 30 08:20 label
total 527
-rw------- 1 root root  1667 Jan 30 08:20 12c6cb72-3ea3-49e7-b381-e0cdfc5e8960.jpg
-rw------- 1 root root  1882 Jan 30 08:20 1e241dc8-8f18-4955-8988-03a0ab49f813.jpg
-rw------- 1 root root  2092 Jan 30 08:20 22e54a62-57a8-4a0a-88c1-4b9758f67651.jpg
-rw------- 1 root root 10710 Jan 30 08:20 316b64c0-55bf-4079-a1c0-d93f461a576f.jpg
-rw------- 1 root root  2092 Jan 30 08:20 37170dd1-2802-4e38-b982-c5d07c64ff67.jpg
-rw------- 1 root root  2067 Jan 30 08:20 4be2025c-09f7-4bb0-b1bd-8e8633e6dec1.jpg
-rw------- 1 root root  1860 Jan 30 08:20 5b562a61-34ad-4f00-9164-d34abb7a38e4.jpg
-rw------- 1 root root  2096 Jan 30 08:20 c9368c55-210d-456c-a5ef-c310e60039ec.jpg
-rw------- 1 root root 14111 Jan 30 08:20 car13.jpg
-rw------- 1 root root  3431 Jan 30 

### 1.1 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [5]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && sudo chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

env: LOCAL_PROJECT_DIR=/ngc_content/
env: CLI=ngccli_cat_linux.zip
--2023-01-30 08:20:41--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 18.65.229.16, 18.65.229.127, 18.65.229.96, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|18.65.229.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41720199 (40M) [application/zip]
Saving to: ‘/ngc_content//ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  39.79M  74.3MB/s    in 0.5s    

2023-01-30 08:20:41 (74.3 MB/s) - ‘/ngc_content//ngccli/ngccli_cat_linux.zip’ saved [41720199/41720199]



In [6]:
!ngc registry model list nvidia/tao/lprnet:*

+-----------------+----------+--------+------------+-----------+------------------+-----------+-----------------+--------------+
| Version         | Accuracy | Epochs | Batch Size | GPU Model | Memory Footprint | File Size | Status          | Created Date |
+-----------------+----------+--------+------------+-----------+------------------+-----------+-----------------+--------------+
| trainable_v1.0  | 99.67    | 120    | 1          | V100      | 221.1            | 221.06 MB | UPLOAD_COMPLETE | Aug 24, 2021 |
| deployable_v1.0 | 99.67    | 120    | 1          | V100      | 110.1            | 110.09 MB | UPLOAD_COMPLETE | Aug 24, 2021 |
+-----------------+----------+--------+------------+-----------+------------------+-----------+-----------------+--------------+


In [7]:
!mkdir -p $EXPERIMENT_DIR/pretrained_lprnet_baseline18/

In [8]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/lprnet:trainable_v1.0 --dest $EXPERIMENT_DIR/pretrained_lprnet_baseline18

Downloaded 221.13 MB in 10s, Download speed: 22.09 MB/s               
--------------------------------------------------------------------------------
   Transfer id: lprnet_vtrainable_v1.0
   Download status: Completed
   Downloaded local path: /content/drive/MyDrive/results/lprnet/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0
   Total files downloaded: 4
   Total downloaded size: 221.13 MB
   Started at: 2023-01-30 08:20:57.539571
   Completed at: 2023-01-30 08:21:07.551506
   Duration taken: 10s
--------------------------------------------------------------------------------


In [9]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0

Check that model is downloaded into dir.
total 226365
-rw------- 1 root root       200 Jan 30 08:21 ch_lp_characters.txt
-rw------- 1 root root 115963904 Jan 30 08:21 ch_lprnet_baseline18_trainable.tlt
-rw------- 1 root root        70 Jan 30 08:21 us_lp_characters.txt
-rw------- 1 root root 115832832 Jan 30 08:21 us_lprnet_baseline18_trainable.tlt


## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [10]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

env: GOOGLE_COLAB=1
Mounted at /content/drive


### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [11]:
import os
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["bash_script"] = "setup_env.sh"
else:
    os.environ["bash_script"] = "setup_env_desktop.sh"

!sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

!sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1,882 kB]
Get:14 http://a

In [12]:
if os.environ.get("PYTHONPATH","") == "":
    os.environ["PYTHONPATH"] = ""
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"
else:
    os.environ["PYTHONPATH"]+=":/home_duplicate/rarunachalam/miniconda3/envs/tf_py_36/lib/python3.6/site-packages/third_party/nvml" # FIX MINICONDA PATH

### 2.4 Reset env variables (Use the same paths which was set in Step 0) <a class="anchor" id="head-2-4"></a>

In [13]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env GPU_INDEX=0

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/nvidia-tao
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise Exception("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/lprnet
%env DATA_DIR=/content/drive/MyDrive/lprnet_data/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/lprnet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

env: TAO_DOCKER_DISABLE=1
env: KEY=nvidia_tlt
env: NUM_GPUS=1
env: GPU_INDEX=0
env: COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/nvidia-tao
env: EXPERIMENT_DIR=/content/drive/MyDrive/results/lprnet
env: DATA_DIR=/content/drive/MyDrive/lprnet_data/
env: SPECS_DIR=/content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/specs
total 2
-rw------- 1 root root   70 Jan 30 08:19 us_lp_characters.txt
-rw------- 1 root root 1137 Jan 30 08:19 tutorial_spec.txt


## 3. Provide training specification <a class="anchor" id="head-3"></a>

* Note the spec $SPEC_DIR/default_sepc.txt is for training on US license plates:
    * the max license plate length is 8;
        * You can change `max_label_length` in `lpr_config` to satisfy your own dataset.
    * the characters of US license plates are: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T, U, V, W, X, Y, Z 
        * You can change `characters_list_file` in `dataset_config` to set your own characters.
        * `characters_list_file` should contain all the characters in dataset. And one character takes one line. 

In [14]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/tutorial_spec.txt
!sed -i "s|TAO_SPEC_DIR|$SPECS_DIR/|g" $SPECS_DIR/tutorial_spec.txt
!cat $SPECS_DIR/tutorial_spec.txt

random_seed: 42
lpr_config {
  hidden_units: 512
  max_label_length: 8
  arch: "baseline"
  nlayers: 18 #setting nlayers to be 10 to use baseline10 model
}
training_config {
  batch_size_per_gpu: 32
  num_epochs: 24
  learning_rate {
  soft_start_annealing_schedule {
    min_learning_rate: 1e-6
    max_learning_rate: 1e-5
    soft_start: 0.001
    annealing: 0.5
  }
  }
  regularizer {
    type: L2
    weight: 5e-4
  }
}
eval_config {
  validation_period_during_training: 5
  batch_size: 1
}
augmentation_config {
    output_width: 96
    output_height: 48
    output_channel: 3
    max_rotate_degree: 5
    rotate_prob: 0.5
    gaussian_kernel_size: 5
    gaussian_kernel_size: 7
    gaussian_kernel_size: 15
    blur_prob: 0.5
    reverse_color_prob: 0.5
    keep_original_prob: 0.3
}
dataset_config {
  data_sources: {
    label_directory_path: "/content/drive/MyDrive/lprnet_data///train/label"
    image_directory_path: "/content/drive/MyDrive/lprnet_data///train/image"
  }
  characters_lis

In [16]:
!cat $SPECS_DIR/us_lp_characters.txt

0
1
2
3
4
5
6
7
8
9
A
B
C
D
E
F
G
H
I
J
K
L
M
N
P
Q
R
S
T
U
V
W
X
Y
Z


## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [15]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_unpruned

In [17]:
print("For multi-GPU, change --gpus based on your machine.")
!tao lprnet train --gpus=1 --gpu_index=$GPU_INDEX \
                  -e $SPECS_DIR/tutorial_spec.txt \
                  -r $EXPERIMENT_DIR/experiment_dir_unpruned \
                  -k $KEY \
                  -m $EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0/us_lprnet_baseline18_trainable.tlt

For multi-GPU, change --gpus based on your machine.
Using TensorFlow backend.
2023-01-30 08:49:59.750062: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-01-30 08:50:05.584527: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.
2023-01-30 08:50:08.334592: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2023-01-30 08:50:08.334789: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x79101c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-01-30 08:50:08.334840: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-01-30 08:50:08.336513: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-30 08:50:08.516095: I t

In [18]:
print("To resume training from a checkpoint, set the -m option to be the .tlt you want to resume from and --initial_epochs to be the epoch index of the resumed checkpoint")
# !tao lprnet train --gpu_index=$GPU_INDEX \
#                   -e $SPECS_DIR/tutorial_spec.txt \
#                   -r $EXPERIMENT_DIR/experiment_dir_unpruned \
#                   -k $KEY \
#                   -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-01.tlt
#                   --initial_epoch 2

To resume training from a checkpoint, set the -m option to be the .tlt you want to resume from and --initial_epochs to be the epoch index of the resumed checkpoint


In [19]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/experiment_dir_unpruned/weights/

Model for each epoch:
---------------------
total 553M
-rw------- 1 root root 111M Jan 30 08:51 lprnet_epoch-05.tlt
-rw------- 1 root root 111M Jan 30 08:51 lprnet_epoch-10.tlt
-rw------- 1 root root 111M Jan 30 08:51 lprnet_epoch-15.tlt
-rw------- 1 root root 111M Jan 30 08:51 lprnet_epoch-20.tlt
-rw------- 1 root root 111M Jan 30 08:51 lprnet_epoch-24.tlt


## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [20]:
!tao lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                     -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                     -k $KEY

Using TensorFlow backend.
2023-01-30 09:00:24.838174: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-01-30 09:00:28.686788: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.
2023-01-30 09:00:31,170 [INFO] root: Starting LPRNet evaluation.
2023-01-30 09:00:31,171 [INFO] iva.lprnet.utils.spec_loader: Merging specification from /content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/specs/tutorial_spec.txt
2023-01-30 09:00:33.080690: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-30 09:00:33.085854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1086] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 09:00:33.086047: I tensorflow/core/common_runtime/gpu/

## 6. Inferences <a class="anchor" id="head-6"></a>
In this section, we run the lprnet inference tool to generate inferences on the trained models and print the results. 

In [21]:
# Running inference for detection on n images
!tao lprnet inference --gpu_index=$GPU_INDEX -i $DATA_DIR/val/image \
                      -e $SPECS_DIR/tutorial_spec.txt \
                      -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                      -k $KEY 

Using TensorFlow backend.
2023-01-30 09:00:54.938683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-01-30 09:00:59.047166: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.
INFO: Log file already exists at /content/drive/MyDrive/results/lprnet/experiment_dir_unpruned/weights/status.json
INFO: Starting LPRNet Inference.
INFO: Merging specification from /content/drive/MyDrive/nvidia-tao/tensorflow/lprnet/specs/tutorial_spec.txt
2023-01-30 09:01:03.351731: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-30 09:01:03.356831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1086] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 09:01:03.357010: I tensorf